In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/xl-airways-france/?sortby=post_date%3ADesc&pagesize=20'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    try:
        review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    
    except:
        review_dict['Overall_rating'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'xl airways france'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/xl_airways_france.csv',index=False)

In [3]:
# Sanity check
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
15,xl airways france,R Dorlen,2017-09-02,✅ Verified Review | Miami to Paris. So readin...,Solo Leisure,Economy Class,7,3,5,4,1,3,5,yes
16,xl airways france,M Lazado,2017-08-31,✅ Verified Review | Paris to Los Angeles. I w...,Couple Leisure,Economy Class,10,3,5,5,5,5,5,yes
17,xl airways france,P Layvar,2017-08-11,✅ Verified Review | San Francisco to Paris CD...,Family Leisure,Economy Class,1,1,5,1,1,1,1,no
18,xl airways france,H Speccisi,2017-08-11,"✅ Verified Review | Paris to San Francisco, t...",Couple Leisure,Economy Class,3,1,3,1,1,4,2,no
19,xl airways france,M Ramiro,2017-07-31,✅ Verified Review | San Francisco to Paris. I...,Couple Leisure,Economy Class,10,3,5,4,3,5,5,yes


In [4]:
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,xl airways france,S Pagouli,2019-09-05,✅ Trip Verified | Paris to San Francisco. I w...,Couple Leisure,Economy Class,9,4,5,3,3,5,5,yes
1,xl airways france,D Meares,2019-08-12,✅ Trip Verified | Paris to San Francisco. Rea...,Family Leisure,Economy Class,8,3,5,5,4,3,5,yes
2,xl airways france,Maria Barrow,2019-08-11,Not Verified | My family and I traveled this ...,Family Leisure,Economy Class,1,1,3,2,1,1,1,no
3,xl airways france,Anthony Kuhn,2019-07-22,Not Verified | Newark to Paris. Worst Airline...,Couple Leisure,Economy Class,1,1,1,1,1,1,1,no
4,xl airways france,J Gadora,2019-07-13,✅ Trip Verified | New York to Paris. They los...,Family Leisure,Economy Class,1,1,1,1,1,1,1,no
